In [28]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split


X, y = fetch_openml('Fashion-MNIST', version=1, return_X_y=True)
# Розділення на навчальний та тестовий набори
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(X_train)
x_test = scaler.transform(X_test)

In [34]:
import numpy as np


# Розділення навчального набору на два: з мітками та без міток
X_labeled, X_unlabeled, y_labeled, y_unlabeled = train_test_split(x_train, y_train, test_size=0.5, random_state=42)
y_unlabeled = y_unlabeled.replace(to_replace=y_unlabeled.values, value=-1)

X_labeled, X_unlabeled, y_labeled, y_unlabeled

/tmp/ipykernel_135228/3967215044.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_unlabeled = y_unlabeled.replace(to_replace=y_unlabeled.values, value=-1)
/tmp/ipykernel_135228/3967215044.py:6: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  y_unlabeled = y_unlabeled.replace(to_replace=y_unlabeled.values, value=-1)


(array([[-0.00796844, -0.02016575, -0.02805079, ..., -0.15864275,
         -0.09123214, -0.03342068],
        [-0.00796844, -0.02016575, -0.02805079, ..., -0.15864275,
         -0.09123214, -0.03342068],
        [-0.00796844, -0.02016575, -0.02805079, ..., -0.15864275,
         -0.09123214, -0.03342068],
        ...,
        [-0.00796844, -0.02016575, -0.02805079, ..., -0.15864275,
         -0.09123214, -0.03342068],
        [-0.00796844, -0.02016575, -0.02805079, ..., -0.15864275,
         -0.09123214, -0.03342068],
        [-0.00796844, -0.02016575, -0.02805079, ..., -0.15864275,
         -0.09123214, -0.03342068]]),
 array([[-0.00796844, -0.02016575, -0.02805079, ..., -0.15864275,
         -0.09123214, -0.03342068],
        [-0.00796844, -0.02016575, -0.02805079, ..., -0.15864275,
         -0.09123214, -0.03342068],
        [-0.00796844, -0.02016575, -0.02805079, ..., -0.15864275,
         -0.09123214, -0.03342068],
        ...,
        [-0.00796844, -0.02016575, -0.02805079, ..., -

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


base_classifier = SVC(C=1.0, kernel='rbf', gamma='scale', probability=True)
base_classifier.fit(X_labeled, y_labeled)

# Оцінка точності класифікатора з мітками на тестовому наборі
y_pred_labeled = base_classifier.predict(x_test)
accuracy_score(y_test, y_pred_labeled)

In [ ]:
from sklearn.semi_supervised import SelfTrainingClassifier, LabelPropagation, LabelSpreading


# Класифікатор самонавчання
self_training_classifier = SelfTrainingClassifier(base_classifier)
X_labeled.shape, y_labeled.shape, X_unlabeled.shape, y_unlabeled.shape
self_training_classifier.fit(X_unlabeled, y_unlabeled)

y_pred_self_training = self_training_classifier.predict(x_test)
accuracy_score(y_test, y_pred_self_training)

X_unlabeled, y_unlabeled

In [ ]:
# x_train_int = X_unlabeled.astype(int)
# y_train_int = y_train.astype(int)

# Класифікатор поширення міток
label_propagation_classifier = LabelPropagation()
label_propagation_classifier.fit(x_train, y_train)

y_pred_label_propagation = label_propagation_classifier.predict(x_test)
accuracy_score(y_test, y_pred_label_propagation)

In [ ]:
# Класифікатор розповсюдження міток
label_spreading_classifier = LabelSpreading()
label_spreading_classifier.fit(x_train, y_train)

y_pred_label_spreading = label_spreading_classifier.predict(x_test)
accuracy_label_spreading = accuracy_score(y_test, y_pred_label_spreading)